In [6]:
import string

# Generate 5x5 key matrix
def generate_key_matrix(key):
    key = key.upper().replace("J", "I")
    matrix = []
    used = set()

    for ch in key:
        if ch.isalpha() and ch not in used:
            matrix.append(ch)
            used.add(ch)

    for ch in string.ascii_uppercase:
        if ch not in used and ch != "J":
            matrix.append(ch)

    return [matrix[i:i+5] for i in range(0, 25, 5)]

# Find position of character in matrix
def find_position(matrix, ch):
    ch = "I" if ch == "J" else ch  # unify J -> I
    for i in range(5):
        for j in range(5):
            if matrix[i][j] == ch:
                return i, j
    raise ValueError(f"Character {ch} not found in matrix")

# Preprocess plaintext into digraphs
def process_text(text):
    text = text.upper().replace("J", "I")
    text = "".join([c for c in text if c.isalpha()])

    pairs = []
    i = 0
    while i < len(text):
        a = text[i]
        b = text[i+1] if i+1 < len(text) else "X"
        if a == b:
            pairs.append(a + "X")
            i += 1
        else:
            pairs.append(a + b)
            i += 2
    return pairs

# Encrypt letters only
def encrypt_playfair(plaintext, matrix):
    letters = "".join([c for c in plaintext.upper().replace("J", "I") if c.isalpha()])
    pairs = process_text(letters)

    encrypted_letters = ""
    for pair in pairs:
        a, b = pair[0], pair[1]
        r1, c1 = find_position(matrix, a)
        r2, c2 = find_position(matrix, b)

        if r1 == r2:  # Same row
            encrypted_letters += matrix[r1][(c1+1) % 5]
            encrypted_letters += matrix[r2][(c2+1) % 5]
        elif c1 == c2:  # Same column
            encrypted_letters += matrix[(r1+1) % 5][c1]
            encrypted_letters += matrix[(r2+1) % 5][c2]
        else:  # Rectangle rule
            encrypted_letters += matrix[r1][c2]
            encrypted_letters += matrix[r2][c1]

    # Reinsert punctuation
    result = ""
    j = 0
    for ch in plaintext:
        if ch.isalpha():
            result += encrypted_letters[j]
            j += 1
        else:
            result += ch
    return result

# Decrypt letters only
def decrypt_playfair(ciphertext, matrix):
    letters = "".join([c for c in ciphertext.upper() if c.isalpha()])
    pairs = [letters[i:i+2] for i in range(0, len(letters), 2)]

    decrypted_letters = ""
    for pair in pairs:
        a, b = pair[0], pair[1]
        r1, c1 = find_position(matrix, a)
        r2, c2 = find_position(matrix, b)

        if r1 == r2:  # Same row
            decrypted_letters += matrix[r1][(c1-1) % 5]
            decrypted_letters += matrix[r2][(c2-1) % 5]
        elif c1 == c2:  # Same column
            decrypted_letters += matrix[(r1-1) % 5][c1]
            decrypted_letters += matrix[(r2-1) % 5][c2]
        else:  # Rectangle rule
            decrypted_letters += matrix[r1][c2]
            decrypted_letters += matrix[r2][c1]

    # Remove filler X
    cleaned = ""
    i = 0
    while i < len(decrypted_letters):
        if (i < len(decrypted_letters)-2 and
            decrypted_letters[i] == decrypted_letters[i+2] and
            decrypted_letters[i+1] == "X"):
            cleaned += decrypted_letters[i]
            i += 2
        else:
            cleaned += decrypted_letters[i]
            i += 1
    if cleaned.endswith("X"):
        cleaned = cleaned[:-1]

    # Reinsert punctuation
    result = ""
    j = 0
    for ch in ciphertext:
        if ch.isalpha():
            result += cleaned[j]
            j += 1
        else:
            result += ch
    return result

# ---------------- Main ----------------
key = input("Enter key: ")
matrix = generate_key_matrix(key)

print("\nKey Matrix:")
for row in matrix:
    print(row)

plaintext = input("\nEnter plaintext: ")
cipher = encrypt_playfair(plaintext, matrix)
print("🔒 Encrypted:", cipher)

plain = decrypt_playfair(cipher, matrix)
print("🔓 Decrypted:", plain)


Enter key:  kamrn



Key Matrix:
['K', 'A', 'M', 'R', 'N']
['B', 'C', 'D', 'E', 'F']
['G', 'H', 'I', 'L', 'O']
['P', 'Q', 'S', 'T', 'U']
['V', 'W', 'X', 'Y', 'Z']



Enter plaintext:  hii this is a book


🔒 Encrypted: ILL SILX SQ M FGGN
🔓 Decrypted: HII THIS IS A BOOK
